In [ ]:
import torch
from src.algo.nn.model.TFT import TemporalFusionTransformer
from src.model.data_module import get_realistic_batch_data
from src.model.util.batch import BatchOutput
batch_data = get_realistic_batch_data('day+indus')
tft = TemporalFusionTransformer(
    input_dim = (6,35),
    hidden_dim = 16,
    ffn_dim = 32,
    num_heads = 4,
    pred_len = 4,
    encoder_layers = 2,
    dropout = 0.1
)

batch_output = BatchOutput.nn_module(tft , batch_data)
batch_output.shape

main path: /Users/mengkjin/workspace/learndl
src.INSTANCE_RECORD can be accessed to check ['trainer', 'account', 'factor']
Load  3 DataBlocks...... finished! Cost 0.27 secs
Align 3 DataBlocks...... finished! Cost 0.92 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
Pre-Norming method of [indus] : {'divlast': False, 'histnorm': False}
static_features: torch.Size([5324, 35])
static_embed: torch.Size([5324, 8])
static_encoded: torch.Size([5324, 30, 16])
static_weights: torch.Size([5324, 30, 8])
static_context: torch.Size([5324, 16])
historical_features: torch.Size([5324, 30, 6])
historical_encoded: torch.Size([5324, 30, 16])
hist_weights: torch.Size([5324, 30, 6])
lstm_input: torch.Size([5324, 30, 16])
encoder_output: torch.Size([5324, 30, 16])
h_n: torch.Size([1, 5324, 16])
c_n: torch.Size([1, 5324, 16])
decoder_input: torch.Size([5324, 4, 16])
decoder_output: torch.Size([5324, 4, 16])
attention_output: torch.Size([5324, 4, 16])
predictions: torch.Size([5324, 4, 3]

{'pred': torch.Size([5324, 1]),
 'predictions': torch.Size([5324, 4, 3]),
 'static_weights': torch.Size([5324, 30, 8]),
 'historical_weights': torch.Size([5324, 30, 6]),
 'future_weights': float}

In [5]:
batch_data.shape

{'x': (torch.Size([5324, 30, 6]), torch.Size([5324, 30, 35])),
 'y': torch.Size([5324, 1])}

In [4]:
batch_output.shape

{'pred': torch.Size([5324, 1]),
 'predictions': torch.Size([5324, 4, 3]),
 'static_weights': torch.Size([5324, 30, 8]),
 'historical_weights': torch.Size([5324, 30, 6]),
 'future_weights': float}

In [6]:
tft.loss(batch_data.y , **batch_output)

tensor([0.3995], grad_fn=<MeanBackward1>)

In [1]:
batch_data.shape

NameError: name 'batch_data' is not defined

In [4]:
batch_data.y.shape , batch_output['predictions'].shape

(torch.Size([5324, 1]), torch.Size([5324, 4, 3]))

In [4]:
batch_data.x[1].shape

torch.Size([5324, 30, 35])

In [2]:
import torch
import torch.nn as nn
from torch import Tensor

class StockWiseTranformer(nn.Module):
    def __init__(self , input_dim , hidden_dim = 4 , ffn_dim = 16 , num_heads = 8 , encoder_layers = 2 , dropout=0.0):
        super().__init__()
        self.fc_att = TimeWiseAttention(hidden_dim,hidden_dim)
        enc_layer  = nn.TransformerEncoderLayer(hidden_dim, num_heads, dim_feedforward=ffn_dim , dropout=dropout , batch_first=True)
        self.trans = nn.TransformerEncoder(enc_layer , encoder_layers)

    def forward(self, x : Tensor , pad_mask = None) -> Tensor:
        if x.isnan().any():
            pad_mask = self.pad_mask_nan(x) if pad_mask is None else (self.pad_mask_nan(x) + pad_mask) > 0
            x = x.nan_to_num()
        if x.dim() != 2: x = self.fc_att(x)
        x = self.trans(x.unsqueeze(0) , src_key_padding_mask = pad_mask)
        return x.squeeze(0)

torch.Size([10, 30, 6])